In [1]:
!pip install requests
!pip install zeep
!pip install openml

In [16]:
import requests
from pprint import pprint
import pandas as pd
from zeep import Client
import logging
import openml
import sqlite3
import xml.etree.ElementTree as ET


# API Number of motor vehicles per 1,000 inhabitants 

Below you can find the end points. We will use the TypedDataSet end point.
```
{
  "odata.metadata":"https://opendata.cbs.nl/ODataApi/OData/85235NED/$metadata","value":[
    {
      "name":"TableInfos","url":"https://opendata.cbs.nl/ODataApi/odata/85235NED/TableInfos"
    },{
      "name":"UntypedDataSet","url":"https://opendata.cbs.nl/ODataApi/odata/85235NED/UntypedDataSet"
    },{
      "name":"TypedDataSet","url":"https://opendata.cbs.nl/ODataApi/odata/85235NED/TypedDataSet"
    },{
      "name":"DataProperties","url":"https://opendata.cbs.nl/ODataApi/odata/85235NED/DataProperties"
    },{
      "name":"CategoryGroups","url":"https://opendata.cbs.nl/ODataApi/odata/85235NED/CategoryGroups"
    },{
      "name":"RegioS","url":"https://opendata.cbs.nl/ODataApi/odata/85235NED/RegioS"
    },{
      "name":"Perioden","url":"https://opendata.cbs.nl/ODataApi/odata/85235NED/Perioden"
    }
  ]
}
```

In [3]:
url = "https://opendata.cbs.nl/ODataApi/odata/85235NED/TypedDataSet"
response= requests.get(url)
users= response.json()
print(users)



{'odata.metadata': 'https://opendata.cbs.nl/ODataApi/OData/85235NED/$metadata#Cbs.OData.WebAPI.TypedDataSet', 'value': [{'ID': 0, 'RegioS': 'NL01  ', 'Perioden': '2019JJ00', 'TotaalWegvoertuigen_1': 11372065, 'TotaalMotorvoertuigen_2': 10199679, 'TotaalPersonenautoS_3': 8442982, 'PersonenautoSRelatief_4': 489, 'InBezitNatuurlijkePersonen_5': 7452085, 'InBezitNatuurlijkePersonenRelatief_6': 431, 'TotaalBedrijfsvoertuigen_7': 2283037, 'TotaalBedrijfsmotorvoertuigen_8': 1110651, 'TotaalAanhangwagensEnOpleggers_9': 1172386, 'TotaalBedrijfsmotorvoertuigen_10': 1110651, 'Bestelauto_11': 903005, 'VrachtautoExclTrekkerVoorOplegger_12': 61652, 'TrekkerVoorOplegger_13': 78788, 'SpeciaalVoertuig_14': 57693, 'Bus_15': 9513, 'TotaalAanhangwagensEnOpleggers_16': 1172386, 'Aanhangwagen_17': 1008412, 'Oplegger_18': 163974, 'TotaalMotorfietsen_19': 646046, 'MotorfietsenRelatief_20': 37}, {'ID': 1, 'RegioS': 'NL01  ', 'Perioden': '2020JJ00', 'TotaalWegvoertuigen_1': 11567203, 'TotaalMotorvoertuigen_2': 

##### In order to make the output above more readable print the following cell

In [4]:
# this will print the typed dataset in json format, so name value pairs
url = "https://opendata.cbs.nl/ODataApi/odata/85235NED/TypedDataSet"
posts = requests.get(url).json()

pprint (posts)





{'odata.metadata': 'https://opendata.cbs.nl/ODataApi/OData/85235NED/$metadata#Cbs.OData.WebAPI.TypedDataSet',
 'value': [{'Aanhangwagen_17': 1008412,
            'Bestelauto_11': 903005,
            'Bus_15': 9513,
            'ID': 0,
            'InBezitNatuurlijkePersonenRelatief_6': 431,
            'InBezitNatuurlijkePersonen_5': 7452085,
            'MotorfietsenRelatief_20': 37,
            'Oplegger_18': 163974,
            'Perioden': '2019JJ00',
            'PersonenautoSRelatief_4': 489,
            'RegioS': 'NL01  ',
            'SpeciaalVoertuig_14': 57693,
            'TotaalAanhangwagensEnOpleggers_16': 1172386,
            'TotaalAanhangwagensEnOpleggers_9': 1172386,
            'TotaalBedrijfsmotorvoertuigen_10': 1110651,
            'TotaalBedrijfsmotorvoertuigen_8': 1110651,
            'TotaalBedrijfsvoertuigen_7': 2283037,
            'TotaalMotorfietsen_19': 646046,
            'TotaalMotorvoertuigen_2': 10199679,
            'TotaalPersonenautoS_3': 8442982,
   

In [5]:
# select columns by first selecting the "value" key from json format
# then specify the specific columns
dfposts = pd.DataFrame(posts["value"])[['Aanhangwagen_17',
            'Bestelauto_11',
            'Bus_15',
            'ID',
            'InBezitNatuurlijkePersonenRelatief_6',
            'InBezitNatuurlijkePersonen_5',
            'MotorfietsenRelatief_20',
            'Oplegger_18',
            'Perioden',
            'PersonenautoSRelatief_4',
            'RegioS',
            'SpeciaalVoertuig_14',
            'TotaalAanhangwagensEnOpleggers_16',
            'TotaalAanhangwagensEnOpleggers_9',
            'TotaalBedrijfsmotorvoertuigen_10',
            'TotaalBedrijfsmotorvoertuigen_8',
            'TotaalBedrijfsvoertuigen_7',
            'TotaalMotorfietsen_19',
            'TotaalMotorvoertuigen_2',
            'TotaalPersonenautoS_3',
            'TotaalWegvoertuigen_1',
            'TrekkerVoorOplegger_13',
            'VrachtautoExclTrekkerVoorOplegger_12']]
dfposts.head()


,Aanhangwagen_17,Bestelauto_11,Bus_15,ID,InBezitNatuurlijkePersonenRelatief_6,InBezitNatuurlijkePersonen_5,MotorfietsenRelatief_20,Oplegger_18,Perioden,PersonenautoSRelatief_4,...,TotaalAanhangwagensEnOpleggers_9,TotaalBedrijfsmotorvoertuigen_10,TotaalBedrijfsmotorvoertuigen_8,TotaalBedrijfsvoertuigen_7,TotaalMotorfietsen_19,TotaalMotorvoertuigen_2,TotaalPersonenautoS_3,TotaalWegvoertuigen_1,TrekkerVoorOplegger_13,VrachtautoExclTrekkerVoorOplegger_12
0,1008412,903005,9513,0,431,7452085,37,163974,2019JJ00,489,...,1172386,1110651,1110651,2283037,646046,10199679,8442982,11372065,78788,61652
1,1023980,927251,9699,1,434,7548770,38,168502,2020JJ00,493,...,1192482,1135943,1135943,2328425,654387,10374721,8584391,11567203,80118,61746
2,1029636,945433,9050,2,440,7683236,38,170014,2021JJ00,497,...,1199650,1151740,1151740,2351390,666597,10504756,8686419,11704406,79932,60831
3,1051486,974792,8532,3,444,7803950,39,177294,2022JJ00,502,...,1228780,1182857,1182857,2411637,677787,10688353,8827709,11917133,82436,60867
4,1069827,989841,8756,4,440,7842473,39,185262,2023JJ00,501,...,1255089,1201061,1201061,2456150,690724,10808892,8917107,12063981,85679,60811


# API Luchtmeetnet

In this part of the code, we will retrieve data from the luchtmeetnet api. We filter through the measurements using ?formula=NO2 , ?formula=PM10 and ?formula=PM25  

In [6]:
url = "https://api.luchtmeetnet.nl/open_api/measurements?formula=NO2" # get data from luchtmeetnet api and filter it so that only NO2 formulas will be retrieved
posts_luchtmeetnet_NO2 = requests.get(url).json()
#pprint(posts_luchtmeetnet_NO2)



In [7]:
url = "https://api.luchtmeetnet.nl/open_api/measurements?formula=PM25" # get data from luchtmeetnet api and filter it so that only PM25 formulas will be retrieved
posts_luchtmeetnet_PM25 = requests.get(url).json()
#pprint(posts_luchtmeetnet_PM25)


In [8]:
url = "https://api.luchtmeetnet.nl/open_api/measurements?formula=PM10" # get data from luchtmeetnet api and filter it so that only PM10 formulas will be retrieved
posts_luchtmeetnet_PM10 = requests.get(url).json()
#pprint(posts_luchtmeetnet_PM10)

We know want to put it in a dataframe 

In [9]:
dfposts_NO2 = pd.DataFrame(posts_luchtmeetnet_NO2["data"])[['formula',
           'station_number',
           'timestamp_measured',
           'value']]
dfposts_NO2.head(10)

,formula,station_number,timestamp_measured,value
0,NO2,NL50002,2025-09-26T20:00:00+00:00,24.00
1,NO2,NL01489,2025-09-26T19:00:00+00:00,28.20
2,NO2,NL10641,2025-09-26T19:00:00+00:00,17.01
3,NO2,NL10929,2025-09-26T19:00:00+00:00,7.62
4,NO2,NL10107,2025-09-26T19:00:00+00:00,14.83
5,NO2,NL10633,2025-09-26T19:00:00+00:00,10.41
6,NO2,NL10738,2025-09-26T19:00:00+00:00,17.88
7,NO2,NL10742,2025-09-26T19:00:00+00:00,33.70
8,NO2,NL49680,2025-09-26T19:00:00+00:00,14.40
9,NO2,NL50010,2025-09-26T19:00:00+00:00,21.00


In [10]:
dfposts_PM25 = pd.DataFrame(posts_luchtmeetnet_PM25["data"])[['formula',
           'station_number',
           'timestamp_measured',
           'value']]
dfposts_PM25.head(10)

,formula,station_number,timestamp_measured,value
0,PM25,NL50012,2025-09-26T20:00:00+00:00,6.00
1,PM25,NL50007,2025-09-26T20:00:00+00:00,12.00
2,PM25,NL50011,2025-09-26T20:00:00+00:00,14.00
3,PM25,NL10450,2025-09-26T19:00:00+00:00,14.58
4,PM25,NL49551,2025-09-26T19:00:00+00:00,6.90
5,PM25,NL54004,2025-09-26T19:00:00+00:00,10.08
6,PM25,NL10938,2025-09-26T19:00:00+00:00,9.19
7,PM25,NL10248,2025-09-26T19:00:00+00:00,15.40
8,PM25,NL50012,2025-09-26T19:00:00+00:00,5.00
9,PM25,NL01487,2025-09-26T19:00:00+00:00,15.00


In [11]:
dfposts_PM10 = pd.DataFrame(posts_luchtmeetnet_PM10["data"])[['formula',
           'station_number',
           'timestamp_measured',
           'value']]
dfposts_PM10.head(10)

,formula,station_number,timestamp_measured,value
0,PM10,NL50011,2025-09-26T20:00:00+00:00,25.00
1,PM10,NL50007,2025-09-26T20:00:00+00:00,20.00
2,PM10,NL50012,2025-09-26T20:00:00+00:00,12.00
3,PM10,NL49557,2025-09-26T19:00:00+00:00,20.80
4,PM10,NL10235,2025-09-26T19:00:00+00:00,17.23
5,PM10,NL10237,2025-09-26T19:00:00+00:00,17.23
6,PM10,NL10929,2025-09-26T19:00:00+00:00,26.19
7,PM10,NL01485,2025-09-26T19:00:00+00:00,19.90
8,PM10,NL49551,2025-09-26T19:00:00+00:00,14.70
9,PM10,NL01493,2025-09-26T19:00:00+00:00,24.30


# API Population density

```
{
  "odata.metadata":"https://opendata.cbs.nl/ODataApi/OData/85984NED/$metadata","value":[
    {
      "name":"TableInfos","url":"https://opendata.cbs.nl/ODataApi/odata/85984NED/TableInfos"
    },{
      "name":"UntypedDataSet","url":"https://opendata.cbs.nl/ODataApi/odata/85984NED/UntypedDataSet"
    },{
      "name":"TypedDataSet","url":"https://opendata.cbs.nl/ODataApi/odata/85984NED/TypedDataSet"
    },{
      "name":"DataProperties","url":"https://opendata.cbs.nl/ODataApi/odata/85984NED/DataProperties"
    },{
      "name":"CategoryGroups","url":"https://opendata.cbs.nl/ODataApi/odata/85984NED/CategoryGroups"
    },{
      "name":"WijkenEnBuurten","url":"https://opendata.cbs.nl/ODataApi/odata/85984NED/WijkenEnBuurten"
    }
  ]
}
```

* https://learn.microsoft.com/en-us/odata/concepts/queryoptions-overview -> explains how to use batches since thi API needs to parse from a very large data source. Since it is a very large data set, we will print the first 9000 rows before we begin with preprocessing

### NOTE: it is still not on the same abstraction level as the other APIs. To fix this, we need to specify codering_3 to only values that start with NL

In [12]:
url_population = "https://opendata.cbs.nl/ODataApi/odata/85984NED/TypedDataSet?$top=9000&$skip=0"
response_population= requests.get(url_population)
users_population= response.json()
posts_population = requests.get(url_population).json()

#pprint (posts_population)

In [13]:
dfposts_population = pd.DataFrame(posts_population["value"])[['Bevolkingsdichtheid_34',
           'Codering_3',
           'Gemeentenaam_1']]
dfposts_population.head(10)

,Bevolkingsdichtheid_34,Codering_3,Gemeentenaam_1
0,533.0,NL00,Nederland
1,94.0,GM1680,Aa en Hunze
2,245.0,WK168000,Aa en Hunze
3,1369.0,BU16800000,Aa en Hunze
4,12.0,BU16800009,Aa en Hunze
5,82.0,WK168001,Aa en Hunze
6,754.0,BU16800100,Aa en Hunze
7,11.0,BU16800109,Aa en Hunze
8,48.0,WK168002,Aa en Hunze
9,573.0,BU16800200,Aa en Hunze


# XML emission zone

In [2]:
!pip install xmltodict

In [14]:


import xml.etree.cElementTree as ET
tree = ET.parse(r'C:\Users\Ratiba Zaid\Downloads\emissiezones.xml\emissiezones.xml')
root = tree.getroot()

print(root.tag)
print(root.attrib)

print(root)


{http://datex2.eu/schema/3/messageContainer}messageContainer
{'modelBaseVersion': '3'}
<Element '{http://datex2.eu/schema/3/messageContainer}messageContainer' at 0x0000025B2554D5E0>
